<a href="https://colab.research.google.com/github/pl-cisco/bot/blob/main/mamba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dziala na T4 !!!!

In [1]:
!pip install causal-conv1d==1.0.0
!pip install mamba-ssm==1.0.1
#!pip install transformers
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.7 MB/s eta 0:00:00
  Created wheel for causal-conv1d: filename=causal_conv1d-1.0.0-cp310-cp310-linux_x86_64.whl size=9116761 sha256=4bbd2c2672ecd02c1e43f8e52552de593099619abc6dda18b2ac650e08110124
  Stored in directory: /root/.cache/pip/wheels/9a/48/f5/eb0c6d6d8e00131eaa57927b537a23832b37e2f01b801d9c5d
Successfully built causal-conv1d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for mamba-ssm: filename=mamba_ssm-1.0.1-cp310-cp310-linux_x86_64.whl size=137567739 sha256=1775b610f76d6bc71ffaa72375df8c0afde52c1c14f1c788ad6afee4290adff2
  Stored in directory: /root/.cache/pip/wheels/08/cf/65/cc589985f9689241fe2c154ce1c60738f58a24e76ce474cc20
Successfully built mamba-ssm
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5

In [2]:
import os
import torch
from transformers import AutoTokenizer
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

device = "cuda"
tokenizer = AutoTokenizer.from_pretrained("havenhq/mamba-chat")
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta").chat_template

model = MambaLMHeadModel.from_pretrained("havenhq/mamba-chat", device="cuda", dtype=torch.float16)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

In [3]:
messages = []
user_message = """
what is Catalyst 9200 ?
Introduction Extend intent-based networking everywhere
Cisco ® Catalyst ® 9200 Series switches extend the power of intent-based networking and Catalyst 9000 hardware and software innovation to a broader set of deployments.
With its family pedigree, Catalyst 9200 Series switches offer simplicity without compromise -it is secure, always on, and IT simplified.
As foundational building blocks for the Cisco Digital Network Architecture, Catalyst 9200 Series switches help customers simplify complexity, optimize IT, and reduce operational costs by leveraging intelligence, automation and human expertise that no other vendor can deliver regardless of where you are in the intent -based networking journey.
Catalyst 9200 Series switches provide security features that protect the integrity of the hardware as well as the software and all data that flows through the switch.
It provides resiliency that keeps your business up and running seamlessly.
Combine that with open APIs of Cisco IOS XE ® and programmability of the UADP ASIC technology, Catalyst 9200 Series switches give you what you need now with investment protection on future innovations.
With full PoE+ capability, power and fan redundancy, stacking bandwidth up to 160 Gbps , modular uplinks, Layer 3 feature support, and cold patching, Catalyst 9200 Series switches are the industry's unparalleled solution with differentiated resiliency and progressive architecture for cost-effective branch-office access.
Catalyst 9200 Series switches also provide operational choice of CLI, Cisco Catalyst Center (formerly Cisco DNA Center) on-premises management, or cloud monitoring for Catalyst on Meraki dashboard.
Product overview
Product highlights

"""

messages.append(dict(role="user",content=user_message))

In [4]:
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")
out = model.generate(input_ids=input_ids, max_length=2000, temperature=0.9, top_p=0.7, eos_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(out)
messages.append(dict(role="assistant",content=decoded[0].split("<|assistant|>\n")[-1]))
print("Model:", decoded[0].split("<|assistant|>\n")[-1])

Model: Catalyst 9200 Series switches are designed to simplify the complexity of networking by providing security features, resiliency, and operational choices that can be customized to fit the needs of any organization. They are designed to be simple to use and manage, with a focus on providing the best possible network experience for customers. With full PoE+ capability, power and fan redundancy, stacking bandwidth up to 160 Gbps, modular uplinks, Layer 3 feature support, and cold patching, Catalyst 9200 Series switches are the industry's unparalleled solution with differentiated resiliency and progressive architecture for cost-effective branch-office access.<|endoftext|>
